<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [2]:
from datetime import datetime

# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

data['연도'] = data['일시'].dt.year
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 연, 월 별 중앙값
monthly_medians = data.groupby('월').median()
yearly_medians = data.groupby('연도').median()
# 연, 월 별 평균값
monthly_means = data.groupby('월').mean()
yearly_means = data.groupby('연도').mean()

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

<ipython-input-2-08e5e0c1571b>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('월').median()
<ipython-input-2-08e5e0c1571b>:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly_medians = data.groupby('연도').median()
<ipython-input-2-08e5e0c1571b>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_means = data.groupby('월').mean()
<ipython-input-2-08e5e0c1571b>:15: FutureWa

In [5]:
# 결측값 대체
data.fillna(monthly_medians, inplace=True)

for column in data.columns:
    if data[column].isnull().any():  # 결측값이 있는 경우에만 처리
        data[column+'_중앙값'] = data.apply(lambda row: monthly_medians.loc[row['월'], column], axis=1)
        data[column].fillna(data[column+'_중앙값'], inplace=True)
        data.drop(column+'_중앙값', axis=1, inplace=True)

In [6]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

import warnings
warnings.filterwarnings(action='ignore')

In [16]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, target)

# print(lr.score(train_poly, target))

from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_poly)

lr_mae = mean_absolute_error(target, lr_pred)

print("훈련 세트 MAE:", lr_mae)

훈련 세트 MAE: 0.3650356700680076


In [17]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_scaled, target)

# print(lr.score(train_poly, target))

from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_scaled)

lr_mae = mean_absolute_error(target, lr_pred)

print("훈련 세트 MAE:", lr_mae)

훈련 세트 MAE: 0.365076480582124


In [9]:
# 차수 조정

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# 다항 회귀 모델 생성 및 학습
def train_polynomial_model(degree):
    model = make_pipeline(
        PolynomialFeatures(degree=degree),
        StandardScaler(),
        LinearRegression()
    )

    # 교차 검증을 통한 성능 평가
    scores = cross_val_score(model, train, target, cv=5, scoring='neg_mean_absolute_error')

    # 교차 검증 성능의 평균을 반환
    return -scores.mean()

# 다양한 차수에 대해 교차 검증을 통해 최적의 차수를 찾음
best_degree = None
best_score = float('inf')

for degree in range(1, 5):  # 적절한 범위로 조정
    score = train_polynomial_model(degree)
    if score < best_score:
        best_score = score
        best_degree = degree

# 최적의 차수 출력
print("Best Polynomial Degree:", best_degree)

Best Polynomial Degree: 1


In [10]:
submission['평균기온'] = lr_pred[:358]

print(submission.head(30))

            일시       평균기온
0   2023-01-01  -1.887615
1   2023-01-02  -2.399307
2   2023-01-03   3.317630
3   2023-01-04   5.729972
4   2023-01-05  -3.861919
5   2023-01-06  -5.578606
6   2023-01-07   0.923217
7   2023-01-08   1.622050
8   2023-01-09   1.994953
9   2023-01-10   1.576975
10  2023-01-11  -1.777259
11  2023-01-12  -3.453881
12  2023-01-13  -3.885791
13  2023-01-14  -2.435463
14  2023-01-15  -5.298518
15  2023-01-16  -4.851758
16  2023-01-17  -5.792945
17  2023-01-18  -3.209024
18  2023-01-19   2.078111
19  2023-01-20  -0.476940
20  2023-01-21  -6.927532
21  2023-01-22  -8.939129
22  2023-01-23 -12.923155
23  2023-01-24 -12.553946
24  2023-01-25  -7.687326
25  2023-01-26  -8.185752
26  2023-01-27 -11.339591
27  2023-01-28 -11.166284
28  2023-01-29  -8.271055
29  2023-01-30  -0.862834


In [11]:
submission.to_csv("DEAGYUL_18_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -1.887615
1  2023-01-02 -2.399307
2  2023-01-03  3.317630
3  2023-01-04  5.729972
4  2023-01-05 -3.861919
